In [1]:
from google.cloud import storage
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime
import ast
import warnings
import pytz
import requests
import time
import datetime as dt
import json
from pandas.io.json import json_normalize
from datetime import date, timedelta
import matplotlib.pyplot as plt
import math
import seaborn as sns
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # default='warn'

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
pd.options.display.float_format = '{:.0f}'.format

In [2]:
df_ads = pd.read_csv("gs://aes-datahub-0001-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_OUTAGE_DATASET_V8.6_categorised.csv")

In [3]:
df_ads = df_ads.loc[:, ~df_ads.columns.str.contains('^Unnamed')]

In [4]:
print(df_ads.shape)
df_ads.head()

(19673, 108)


,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DOWNSTREAM_KVA_VAL,KVA_VAL,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,PRIM_OCCURN_DESC,SECO_OCCURN_DESC,TERT_OCCURN_DESC,PRIM_CAUSE_DESC,SECO_CAUSE_DESC,LAT,LONG,ZONE,LIVE_OUTAGE,UNIT,Min_Distance,Marker_Location,CLOUDCOVER,DEWPOINT,HUMIDITY,ICON,NEAREST-STATION,PRECIPINTENSITY,PRECIPINTENSITYMAX,PRECIPTYPE,PRESSURE,TEMPERATUREMAX,TEMPERATUREMIN,VISIBILITY,WINDBEARING,WINDGUST,WINDSPEED,clusters,Hour_Sin,Hour_Cos,MAJ_OTG_ID,STORM_END,STORM_START,LAPSE_STORM_START_CREATIONDATETIME,RANK_SUBSEQUENT_MAJ_OTG_ID,WIND_DIRECTION,SEASON,CAT_TTR
0,2001317842,2000391296,648-B/72,STORM,2007-02-24 20:02:00,2007-02-24 21:45:00,1702,ASWITCH,170,183,1282,4,2007-02-24 21:32:00,1282,6101,6101,0,103,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,SUBSTATION,LOCK OUT,NaN,MISCELLANEOUS,NO CAUSE-PATROLLED,40,-86,2,8,nan,6,Marker12,1,19,1,snow,4,0,0,snow,1019,35,22,9,102,28,8,Cluster3.0,-1,0,20070001001,2007-02-24 22:49:02,2007-02-24 20:02:00,0,1,S-E-E,WINTER,0-120
1,2001317865,2000391302,237--/42,STORM,2007-02-24 20:18:26,2007-02-24 21:50:00,1402,FUSE,140,2,4,0,2007-02-24 22:48:26,4,100,100,0,92,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,FUSE,FUSE BLOWN,NaN,TREE,ON PRIMARY,40,-86,3,8,nan,5,Marker3,1,18,1,snow,2,0,0,snow,1019,35,23,9,103,27,7,Cluster3.0,-1,0,20070001001,2007-02-24 22:49:02,2007-02-24 20:02:00,16,2,S-E-E,WINTER,0-120
2,2001317856,2000391313,M15-U/168,STORM,2007-02-24 20:31:55,2007-02-25 00:23:42,7001,1TBOH,700,1,1,0,2007-02-25 00:01:55,1,15,15,0,232,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,MONROVIA,FUSE,DEFECTIVE FUSE,NaN,MISCELLANEOUS,OTHER,40,-87,1,9,nan,2,Marker19,1,21,1,rain,11,0,0,rain,1018,38,24,9,101,24,8,Cluster7.0,-1,0,20070001001,2007-02-24 22:49:02,2007-02-24 20:02:00,30,3,S-E-E,WINTER,120-240
3,2001317899,2000391321,495-A/106,STORM,2007-02-24 20:56:03,2007-02-25 00:30:00,1257,FUSE,125,133,271,2,2007-02-24 23:26:03,271,504,504,0,214,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,INDIANAPOLIS,FUSE,FUSE BLOWN,NaN,O.H. EQUIPMENT,BAD OR BROKEN INSULATOR,40,-86,4,10,nan,5,Marker11,1,19,1,snow,5,0,0,snow,1019,36,22,9,104,30,10,Cluster3.0,-1,0,20070001001,2007-02-24 22:49:02,2007-02-24 20:02:00,54,4,S-E-E,WINTER,120-240
4,2001317873,2000391329,801QA/2,STORM,2007-02-24 20:59:10,2007-02-24 21:15:02,7056,FUSE,705,4,70,1,2007-02-24 23:29:10,70,870,870,0,16,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,MOORESVILLE,POLE BURNING,POLE BURNING,NaN,O.H. EQUIPMENT,BAD CUTOUT OR BARREL,40,-86,1,11,nan,6,Marker18,1,19,1,snow,12,0,0,snow,1019,36,22,9,104,27,9,Cluster3.0,-1,0,20070001001,2007-02-24 22:49:02,2007-02-24 20:02:00,57,5,S-E-E,WINTER,0-120

In [5]:
datetime_columns = ['STORM_START', 'STORM_END', 'ENERGIZED_DATETIME']
df_ads[datetime_columns] = df_ads[datetime_columns].apply(pd.to_datetime, errors='coerce')
ads = df_ads.groupby(['MAJ_OTG_ID'], as_index=False).agg({'OUTAGE_ID' : 'count',
                                                          'CUST_QTY' : 'sum', 
                                                          'STORM_START' : 'min',
                                                          'STORM_END' : 'max', 
                                                          'ENERGIZED_DATETIME' : 'max'})
ads.head()

,MAJ_OTG_ID,ENERGIZED_DATETIME,CUST_QTY,STORM_START,STORM_END,OUTAGE_ID
0,20070001001,2007-02-25 00:45:25,2491,2007-02-24 20:02:00,2007-02-24 22:49:02,9
1,20070002001,2007-04-03 22:16:20,693,2007-04-03 15:44:00,2007-04-03 20:09:18,14
2,20070003001,2007-04-12 06:15:35,5196,2007-04-11 18:45:00,2007-04-11 21:11:56,21
3,20070004001,2007-06-03 17:02:58,1719,2007-06-02 19:18:55,2007-06-03 12:36:45,28
4,20070005001,2007-06-08 05:00:00,1544,2007-06-07 15:06:16,2007-06-08 02:28:15,28


In [6]:
marker1=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker1_storm_data.csv')
marker2=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker2_storm_data.csv')
marker3=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker3_storm_data.csv')
marker4=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker4_storm_data.csv')
marker5=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker5_storm_data.csv')
marker6=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker6_storm_data.csv')
marker7=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker7_storm_data.csv')
marker8=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker8_storm_data.csv')
marker9=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker9_storm_data.csv')
marker10=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker10_storm_data.csv')
marker11=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker11_storm_data.csv')
marker12=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker12_storm_data.csv')
marker13=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker13_storm_data.csv')
marker14=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker14_storm_data.csv')
marker15=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker15_storm_data.csv')
marker16=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker16_storm_data.csv')
marker17=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker17_storm_data.csv')
marker18=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker18_storm_data.csv')
marker19=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker19_storm_data.csv')
marker20=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker20_storm_data.csv')

In [7]:
# Select and keep only necessary columns 

marker1 = marker1[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker2 = marker2[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker3 = marker3[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker4 = marker4[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker5 = marker5[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker6 = marker6[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker7 = marker7[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker8 = marker8[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker9 = marker9[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker10 = marker10[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker11 = marker11[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker12 = marker12[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker13 = marker13[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker14 = marker14[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker15 = marker15[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker16 = marker16[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker17 = marker17[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker18 = marker18[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker19 = marker19[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

marker20 = marker20[['Date', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'precipAccumulation', 'precipIntensity', 'precipIntensityMax',
                   'precipProbability', 'precipType', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]

In [8]:
location = ['MARKER1', 'MARKER2', 'MARKER3', 'MARKER4', 'MARKER5', 'MARKER6', 'MARKER7', 'MARKER8',
            'MARKER9', 'MARKER10', 'MARKER11', 'MARKER12', 'MARKER13', 'MARKER14', 'MARKER15', 'MARKER16',
            'MARKER17', 'MARKER18', 'MARKER19', 'MARKER20']

i = 0 

marker1.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)


i = i + 1
marker2.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker3.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker4.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)


i = i + 1
marker5.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)


i = i + 1
marker6.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker7.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker8.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker9.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker10.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker11.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker12.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)


i = i + 1
marker13.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker14.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)


i = i + 1
marker15.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker16.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker17.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker18.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

i = i + 1
marker19.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)


i = i + 1
marker20.rename(columns = {"apparentTemperatureMax" : location[i]+"_apparentTemperatureMax",
                          "apparentTemperatureMin": location[i]+"_apparentTemperatureMin",
                          "cloudCover" : location[i]+"_cloudCover",
                          "dewPoint" : location[i]+"_dewPoint",
                          "humidity" : location[i]+"_humidity",
                          "icon" : location[i]+"_icon",
                          "precipAccumulation" : location[i]+"_precipAccumulation",
                          "precipIntensity" : location[i]+"_precipIntensity",
                          "precipIntensityMax" : location[i]+"_precipIntensityMax",
                          "precipProbability" : location[i]+"_precipProbability",
                          "precipType" : location[i]+"_precipType",
                          "pressure" : location[i]+"_pressure",
                          "temperatureMax" : location[i]+"_temperatureMax",
                          "temperatureMin" : location[i]+"_temperatureMin",
                          "visibility" : location[i]+"_visibility",
                          "windBearing" : location[i]+"_windBearing",
                          "windGust" : location[i]+"_windGust",
                          "windSpeed" : location[i]+"_windSpeed"}, inplace=True)

In [9]:
ads.head()

,MAJ_OTG_ID,ENERGIZED_DATETIME,CUST_QTY,STORM_START,STORM_END,OUTAGE_ID
0,20070001001,2007-02-25 00:45:25,2491,2007-02-24 20:02:00,2007-02-24 22:49:02,9
1,20070002001,2007-04-03 22:16:20,693,2007-04-03 15:44:00,2007-04-03 20:09:18,14
2,20070003001,2007-04-12 06:15:35,5196,2007-04-11 18:45:00,2007-04-11 21:11:56,21
3,20070004001,2007-06-03 17:02:58,1719,2007-06-02 19:18:55,2007-06-03 12:36:45,28
4,20070005001,2007-06-08 05:00:00,1544,2007-06-07 15:06:16,2007-06-08 02:28:15,28


In [10]:
ads['STORM_START'] = pd.to_datetime(ads['STORM_START'], errors='coerce')
ads['Date'] = ads['STORM_START'].dt.date
ads['Date'] = pd.to_datetime(ads['Date'])
marker1['Date'] = pd.to_datetime(marker1['Date'])
marker2['Date'] = pd.to_datetime(marker2['Date'])
marker3['Date'] = pd.to_datetime(marker3['Date'])
marker4['Date'] = pd.to_datetime(marker4['Date'])
marker5['Date'] = pd.to_datetime(marker5['Date'])
marker6['Date'] = pd.to_datetime(marker6['Date'])
marker7['Date'] = pd.to_datetime(marker7['Date'])
marker8['Date'] = pd.to_datetime(marker8['Date'])
marker9['Date'] = pd.to_datetime(marker9['Date'])
marker10['Date'] = pd.to_datetime(marker10['Date'])
marker11['Date'] = pd.to_datetime(marker11['Date'])
marker12['Date'] = pd.to_datetime(marker12['Date'])
marker13['Date'] = pd.to_datetime(marker13['Date'])
marker14['Date'] = pd.to_datetime(marker14['Date'])
marker15['Date'] = pd.to_datetime(marker15['Date'])
marker16['Date'] = pd.to_datetime(marker16['Date'])
marker17['Date'] = pd.to_datetime(marker17['Date'])
marker18['Date'] = pd.to_datetime(marker18['Date'])
marker19['Date'] = pd.to_datetime(marker19['Date'])
marker20['Date'] = pd.to_datetime(marker20['Date'])



final_ads  = pd.merge(ads, marker1, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker2, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker3, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker4, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker5, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker6, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker7, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker8, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker9, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker10, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker11, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker12, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker13, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker14, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker15, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker16, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker17, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker18, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker19, how='left', on=['Date'])
final_ads  = pd.merge(final_ads, marker20, how='left', on=['Date'])

In [11]:
print(final_ads.shape)

(249, 367)


In [18]:
final_ads.head()

,MAJ_OTG_ID,CUST_QTY,NO_OF_OUTAGES,MARKER1_apparentTemperatureMax,MARKER1_apparentTemperatureMin,MARKER1_cloudCover,MARKER1_dewPoint,MARKER1_humidity,MARKER1_icon,MARKER1_precipIntensity,MARKER1_precipIntensityMax,MARKER1_precipProbability,MARKER1_precipType,MARKER1_pressure,MARKER1_temperatureMax,MARKER1_temperatureMin,MARKER1_visibility,MARKER1_windBearing,MARKER1_windGust,MARKER1_windSpeed,MARKER2_apparentTemperatureMax,MARKER2_apparentTemperatureMin,MARKER2_cloudCover,MARKER2_dewPoint,MARKER2_humidity,MARKER2_icon,MARKER2_precipIntensity,MARKER2_precipIntensityMax,MARKER2_precipProbability,MARKER2_precipType,MARKER2_pressure,MARKER2_temperatureMax,MARKER2_temperatureMin,MARKER2_visibility,MARKER2_windBearing,MARKER2_windGust,MARKER2_windSpeed,MARKER3_apparentTemperatureMax,MARKER3_apparentTemperatureMin,MARKER3_cloudCover,MARKER3_dewPoint,MARKER3_humidity,MARKER3_icon,MARKER3_precipIntensity,MARKER3_precipIntensityMax,MARKER3_precipProbability,MARKER3_precipType,MARKER3_pressure,MARKER3_temperatureMax,MARKER3_temperatureMin,MARKER3_visibility,MARKER3_windBearing,MARKER3_windGust,MARKER3_windSpeed,MARKER4_apparentTemperatureMax,MARKER4_apparentTemperatureMin,MARKER4_cloudCover,MARKER4_dewPoint,MARKER4_humidity,MARKER4_icon,MARKER4_precipIntensity,MARKER4_precipIntensityMax,MARKER4_precipProbability,MARKER4_precipType,MARKER4_pressure,MARKER4_temperatureMax,MARKER4_temperatureMin,MARKER4_visibility,MARKER4_windBearing,MARKER4_windGust,MARKER4_windSpeed,MARKER5_apparentTemperatureMax,MARKER5_apparentTemperatureMin,MARKER5_cloudCover,MARKER5_dewPoint,MARKER5_humidity,MARKER5_icon,MARKER5_precipIntensity,MARKER5_precipIntensityMax,MARKER5_precipProbability,MARKER5_precipType,MARKER5_pressure,MARKER5_temperatureMax,MARKER5_temperatureMin,MARKER5_visibility,MARKER5_windBearing,MARKER5_windGust,MARKER5_windSpeed,MARKER6_apparentTemperatureMax,MARKER6_apparentTemperatureMin,MARKER6_cloudCover,MARKER6_dewPoint,MARKER6_humidity,MARKER6_icon,MARKER6_precipIntensity,MARKER6_precipIntensityMax,MARKER6_precipProbability,MARKER6_precipType,MARKER6_pressure,MARKER6_temperatureMax,MARKER6_temperatureMin,MARKER6_visibility,MARKER6_windBearing,MARKER6_windGust,MARKER6_windSpeed,MARKER7_apparentTemperatureMax,MARKER7_apparentTemperatureMin,MARKER7_cloudCover,MARKER7_dewPoint,MARKER7_humidity,MARKER7_icon,MARKER7_precipIntensity,MARKER7_precipIntensityMax,MARKER7_precipProbability,MARKER7_precipType,MARKER7_pressure,MARKER7_temperatureMax,MARKER7_temperatureMin,MARKER7_visibility,MARKER7_windBearing,MARKER7_windGust,MARKER7_windSpeed,MARKER8_apparentTemperatureMax,MARKER8_apparentTemperatureMin,MARKER8_cloudCover,MARKER8_dewPoint,MARKER8_humidity,MARKER8_icon,MARKER8_precipIntensity,MARKER8_precipIntensityMax,MARKER8_precipProbability,MARKER8_precipType,MARKER8_pressure,MARKER8_temperatureMax,MARKER8_temperatureMin,MARKER8_visibility,MARKER8_windBearing,MARKER8_windGust,MARKER8_windSpeed,MARKER9_apparentTemperatureMax,MARKER9_apparentTemperatureMin,MARKER9_cloudCover,MARKER9_dewPoint,MARKER9_humidity,MARKER9_icon,MARKER9_precipIntensity,MARKER9_precipIntensityMax,MARKER9_precipProbability,MARKER9_precipType,MARKER9_pressure,MARKER9_temperatureMax,MARKER9_temperatureMin,MARKER9_visibility,MARKER9_windBearing,MARKER9_windGust,MARKER9_windSpeed,MARKER10_apparentTemperatureMax,MARKER10_apparentTemperatureMin,MARKER10_cloudCover,MARKER10_dewPoint,MARKER10_humidity,MARKER10_icon,MARKER10_precipIntensity,MARKER10_precipIntensityMax,MARKER10_precipProbability,MARKER10_precipType,MARKER10_pressure,MARKER10_temperatureMax,MARKER10_temperatureMin,MARKER10_visibility,MARKER10_windBearing,MARKER10_windGust,MARKER10_windSpeed,MARKER11_apparentTemperatureMax,MARKER11_apparentTemperatureMin,MARKER11_cloudCover,MARKER11_dewPoint,MARKER11_humidity,MARKER11_icon,MARKER11_precipIntensity,MARKER11_precipIntensityMax,MARKER11_precipProbability,MARKER11_precipType,MARKER11_pressure,MARKER11_temperatureMax,MARKER11_temperatureMin,MARKER11_visibility,MARKER11_windBeari

In [13]:
final_ads.rename(columns = {'STORM_START' : 'FIRST_CREATION_DATETIME', 'STORM_END' : 'LAST_CREATION_DATETIME', 'OUTAGE_ID' : 'NO_OF_OUTAGES'}, inplace=True)

In [14]:
final_ads = final_ads[final_ads.columns.drop(list(final_ads.filter(regex='_precipAccumulation')))]

In [15]:
final_ads['FIRST_CREATION_DATETIME'] = pd.to_datetime(final_ads['FIRST_CREATION_DATETIME'], errors='coerce')
final_ads['LAST_CREATION_DATETIME'] = pd.to_datetime(final_ads['LAST_CREATION_DATETIME'], errors='coerce')
final_ads['ENERGIZED_DATETIME'] = pd.to_datetime(final_ads['ENERGIZED_DATETIME'], errors='coerce')

final_ads['OUTAGE_DURATION'] = (final_ads.LAST_CREATION_DATETIME - final_ads.FIRST_CREATION_DATETIME).dt.total_seconds().div(3660).round(2)
final_ads['OUTAGED_RECOVERY'] = (final_ads.ENERGIZED_DATETIME - final_ads.FIRST_CREATION_DATETIME).dt.total_seconds().div(3660).round(2)

In [17]:
final_ads.drop(['Date', 'ENERGIZED_DATETIME', 'LAST_CREATION_DATETIME', 'FIRST_CREATION_DATETIME'], axis=1, inplace=True)

In [19]:
print(list(final_ads.columns))

['MAJ_OTG_ID', 'CUST_QTY', 'NO_OF_OUTAGES', 'MARKER1_apparentTemperatureMax', 'MARKER1_apparentTemperatureMin', 'MARKER1_cloudCover', 'MARKER1_dewPoint', 'MARKER1_humidity', 'MARKER1_icon', 'MARKER1_precipIntensity', 'MARKER1_precipIntensityMax', 'MARKER1_precipProbability', 'MARKER1_precipType', 'MARKER1_pressure', 'MARKER1_temperatureMax', 'MARKER1_temperatureMin', 'MARKER1_visibility', 'MARKER1_windBearing', 'MARKER1_windGust', 'MARKER1_windSpeed', 'MARKER2_apparentTemperatureMax', 'MARKER2_apparentTemperatureMin', 'MARKER2_cloudCover', 'MARKER2_dewPoint', 'MARKER2_humidity', 'MARKER2_icon', 'MARKER2_precipIntensity', 'MARKER2_precipIntensityMax', 'MARKER2_precipProbability', 'MARKER2_precipType', 'MARKER2_pressure', 'MARKER2_temperatureMax', 'MARKER2_temperatureMin', 'MARKER2_visibility', 'MARKER2_windBearing', 'MARKER2_windGust', 'MARKER2_windSpeed', 'MARKER3_apparentTemperatureMax', 'MARKER3_apparentTemperatureMin', 'MARKER3_cloudCover', 'MARKER3_dewPoint', 'MARKER3_humidity', 'M

In [20]:
final_ads = final_ads[['MAJ_OTG_ID', 'MARKER1_apparentTemperatureMax', 'MARKER1_apparentTemperatureMin',
                       'MARKER1_cloudCover', 'MARKER1_dewPoint', 'MARKER1_humidity', 'MARKER1_icon', 'MARKER1_precipIntensity', 
                       'MARKER1_precipIntensityMax', 'MARKER1_precipProbability', 'MARKER1_precipType', 'MARKER1_pressure', 
                       'MARKER1_temperatureMax', 'MARKER1_temperatureMin', 'MARKER1_visibility', 'MARKER1_windBearing', 
                       'MARKER1_windGust', 'MARKER1_windSpeed', 'MARKER2_apparentTemperatureMax', 'MARKER2_apparentTemperatureMin',
                       'MARKER2_cloudCover', 'MARKER2_dewPoint', 'MARKER2_humidity', 'MARKER2_icon', 'MARKER2_precipIntensity', 
                       'MARKER2_precipIntensityMax', 'MARKER2_precipProbability', 'MARKER2_precipType', 'MARKER2_pressure', 
                       'MARKER2_temperatureMax', 'MARKER2_temperatureMin', 'MARKER2_visibility', 'MARKER2_windBearing', 
                       'MARKER2_windGust', 'MARKER2_windSpeed', 'MARKER3_apparentTemperatureMax', 'MARKER3_apparentTemperatureMin',
                       'MARKER3_cloudCover', 'MARKER3_dewPoint', 'MARKER3_humidity', 'MARKER3_icon', 'MARKER3_precipIntensity',
                       'MARKER3_precipIntensityMax', 'MARKER3_precipProbability', 'MARKER3_precipType', 'MARKER3_pressure',
                       'MARKER3_temperatureMax', 'MARKER3_temperatureMin', 'MARKER3_visibility', 'MARKER3_windBearing',
                       'MARKER3_windGust', 'MARKER3_windSpeed', 'MARKER4_apparentTemperatureMax', 'MARKER4_apparentTemperatureMin',
                       'MARKER4_cloudCover', 'MARKER4_dewPoint', 'MARKER4_humidity', 'MARKER4_icon', 'MARKER4_precipIntensity',
                       'MARKER4_precipIntensityMax', 'MARKER4_precipProbability', 'MARKER4_precipType', 'MARKER4_pressure',
                       'MARKER4_temperatureMax', 'MARKER4_temperatureMin', 'MARKER4_visibility', 'MARKER4_windBearing',
                       'MARKER4_windGust', 'MARKER4_windSpeed', 'MARKER5_apparentTemperatureMax', 'MARKER5_apparentTemperatureMin',
                       'MARKER5_cloudCover', 'MARKER5_dewPoint', 'MARKER5_humidity', 'MARKER5_icon', 'MARKER5_precipIntensity', 
                       'MARKER5_precipIntensityMax', 'MARKER5_precipProbability', 'MARKER5_precipType', 'MARKER5_pressure', 
                       'MARKER5_temperatureMax', 'MARKER5_temperatureMin', 'MARKER5_visibility', 'MARKER5_windBearing', 'MARKER5_windGust',
                       'MARKER5_windSpeed', 'MARKER6_apparentTemperatureMax', 'MARKER6_apparentTemperatureMin', 'MARKER6_cloudCover',
                       'MARKER6_dewPoint', 'MARKER6_humidity', 'MARKER6_icon', 'MARKER6_precipIntensity', 'MARKER6_precipIntensityMax',
                       'MARKER6_precipProbability', 'MARKER6_precipType', 'MARKER6_pressure', 'MARKER6_temperatureMax', 'MARKER6_temperatureMin',
                       'MARKER6_visibility', 'MARKER6_windBearing', 'MARKER6_windGust', 'MARKER6_windSpeed', 'MARKER7_apparentTemperatureMax',
                       'MARKER7_apparentTemperatureMin', 'MARKER7_cloudCover', 'MARKER7_dewPoint', 'MARKER7_humidity', 'MARKER7_icon',
                       'MARKER7_precipIntensity', 'MARKER7_precipIntensityMax', 'MARKER7_precipProbability', 'MARKER7_precipType', 
                       'MARKER7_pressure', 'MARKER7_temperatureMax', 'MARKER7_temperatureMin', 'MARKER7_visibility', 'MARKER7_windBearing',
                       'MARKER7_windGust', 'MARKER7_windSpeed', 'MARKER8_apparentTemperatureMax', 'MARKER8_apparentTemperatureMin', 'MARKER8_cloudCover',
                       'MARKER8_dewPoint', 'MARKER8_humidity', 'MARKER8_icon', 'MARKER8_precipIntensity', 'MARKER8_precipIntensityMax', 
                       'MARKER8_precipProbability', 'MARKER8_precipType', 'MARKER8_pressure', 'MARKER8_temperatureMax', 'MARKER8_temperatureMin',
                       'MARKER8_visibility', 'MARKER8_windBearing', 'MARKER8_windGust', 'MARKER8_windSpeed', 'MARKER9_apparentTemperatureMax',
                       'MARKER9_apparentTemperatureMin', 'MARKER9_cloudCover', 'MARKER9_dewPoint', 'MARKER9_humidity', 'MARKER9_icon',
                       'MARKER9_precipIntensity', 'MARKER9_precipIntensityMax', 'MARKER9_precipProbability', 'MARKER9_precipType',
                       'MARKER9_pressure', 'MARKER9_temperatureMax', 'MARKER9_temperatureMin', 'MARKER9_visibility', 'MARKER9_windBearing', 
                       'MARKER9_windGust', 'MARKER9_windSpeed', 'MARKER10_apparentTemperatureMax', 'MARKER10_apparentTemperatureMin',
                       'MARKER10_cloudCover', 'MARKER10_dewPoint', 'MARKER10_humidity', 'MARKER10_icon', 'MARKER10_precipIntensity',
                       'MARKER10_precipIntensityMax', 'MARKER10_precipProbability', 'MARKER10_precipType', 'MARKER10_pressure',
                       'MARKER10_temperatureMax', 'MARKER10_temperatureMin', 'MARKER10_visibility', 'MARKER10_windBearing',
                       'MARKER10_windGust', 'MARKER10_windSpeed', 'MARKER11_apparentTemperatureMax', 'MARKER11_apparentTemperatureMin', 
                       'MARKER11_cloudCover', 'MARKER11_dewPoint', 'MARKER11_humidity', 'MARKER11_icon', 'MARKER11_precipIntensity', 
                       'MARKER11_precipIntensityMax', 'MARKER11_precipProbability', 'MARKER11_precipType', 'MARKER11_pressure', 
                       'MARKER11_temperatureMax', 'MARKER11_temperatureMin', 'MARKER11_visibility', 'MARKER11_windBearing', 
                       'MARKER11_windGust', 'MARKER11_windSpeed', 'MARKER12_apparentTemperatureMax', 'MARKER12_apparentTemperatureMin',
                       'MARKER12_cloudCover', 'MARKER12_dewPoint', 'MARKER12_humidity', 'MARKER12_icon', 'MARKER12_precipIntensity', 
                       'MARKER12_precipIntensityMax', 'MARKER12_precipProbability', 'MARKER12_precipType', 'MARKER12_pressure', 
                       'MARKER12_temperatureMax', 'MARKER12_temperatureMin', 'MARKER12_visibility', 'MARKER12_windBearing', 
                       'MARKER12_windGust', 'MARKER12_windSpeed', 'MARKER13_apparentTemperatureMax', 'MARKER13_apparentTemperatureMin', 
                       'MARKER13_cloudCover', 'MARKER13_dewPoint', 'MARKER13_humidity', 'MARKER13_icon', 'MARKER13_precipIntensity', 
                       'MARKER13_precipIntensityMax', 'MARKER13_precipProbability', 'MARKER13_precipType', 'MARKER13_pressure', 
                       'MARKER13_temperatureMax', 'MARKER13_temperatureMin', 'MARKER13_visibility', 'MARKER13_windBearing', 'MARKER13_windGust',
                       'MARKER13_windSpeed', 'MARKER14_apparentTemperatureMax', 'MARKER14_apparentTemperatureMin', 'MARKER14_cloudCover', 
                       'MARKER14_dewPoint', 'MARKER14_humidity', 'MARKER14_icon', 'MARKER14_precipIntensity', 'MARKER14_precipIntensityMax',
                       'MARKER14_precipProbability', 'MARKER14_precipType', 'MARKER14_pressure', 'MARKER14_temperatureMax', 'MARKER14_temperatureMin',
                       'MARKER14_visibility', 'MARKER14_windBearing', 'MARKER14_windGust', 'MARKER14_windSpeed', 'MARKER15_apparentTemperatureMax',
                       'MARKER15_apparentTemperatureMin', 'MARKER15_cloudCover', 'MARKER15_dewPoint', 'MARKER15_humidity', 'MARKER15_icon',
                       'MARKER15_precipIntensity', 'MARKER15_precipIntensityMax', 'MARKER15_precipProbability', 'MARKER15_precipType',
                       'MARKER15_pressure', 'MARKER15_temperatureMax', 'MARKER15_temperatureMin', 'MARKER15_visibility', 'MARKER15_windBearing', 
                       'MARKER15_windGust', 'MARKER15_windSpeed', 'MARKER16_apparentTemperatureMax', 'MARKER16_apparentTemperatureMin',
                       'MARKER16_cloudCover', 'MARKER16_dewPoint', 'MARKER16_humidity', 'MARKER16_icon', 'MARKER16_precipIntensity',
                       'MARKER16_precipIntensityMax', 'MARKER16_precipProbability', 'MARKER16_precipType', 'MARKER16_pressure', 
                       'MARKER16_temperatureMax', 'MARKER16_temperatureMin', 'MARKER16_visibility', 'MARKER16_windBearing', 'MARKER16_windGust',
                       'MARKER16_windSpeed', 'MARKER17_apparentTemperatureMax', 'MARKER17_apparentTemperatureMin', 'MARKER17_cloudCover',
                       'MARKER17_dewPoint', 'MARKER17_humidity', 'MARKER17_icon', 'MARKER17_precipIntensity', 'MARKER17_precipIntensityMax', 
                       'MARKER17_precipProbability', 'MARKER17_precipType', 'MARKER17_pressure', 'MARKER17_temperatureMax', 'MARKER17_temperatureMin',
                       'MARKER17_visibility', 'MARKER17_windBearing', 'MARKER17_windGust', 'MARKER17_windSpeed', 'MARKER18_apparentTemperatureMax',
                       'MARKER18_apparentTemperatureMin', 'MARKER18_cloudCover', 'MARKER18_dewPoint', 'MARKER18_humidity', 'MARKER18_icon',
                       'MARKER18_precipIntensity', 'MARKER18_precipIntensityMax', 'MARKER18_precipProbability', 'MARKER18_precipType', 
                       'MARKER18_pressure', 'MARKER18_temperatureMax', 'MARKER18_temperatureMin', 'MARKER18_visibility', 'MARKER18_windBearing',
                       'MARKER18_windGust', 'MARKER18_windSpeed', 'MARKER19_apparentTemperatureMax', 'MARKER19_apparentTemperatureMin',
                       'MARKER19_cloudCover', 'MARKER19_dewPoint', 'MARKER19_humidity', 'MARKER19_icon', 'MARKER19_precipIntensity',
                       'MARKER19_precipIntensityMax', 'MARKER19_precipProbability', 'MARKER19_precipType', 'MARKER19_pressure', 'MARKER19_temperatureMax',
                       'MARKER19_temperatureMin', 'MARKER19_visibility', 'MARKER19_windBearing', 'MARKER19_windGust', 'MARKER19_windSpeed',
                       'MARKER20_apparentTemperatureMax', 'MARKER20_apparentTemperatureMin', 'MARKER20_cloudCover', 'MARKER20_dewPoint', 
                       'MARKER20_humidity', 'MARKER20_icon', 'MARKER20_precipIntensity', 'MARKER20_precipIntensityMax', 'MARKER20_precipProbability',
                       'MARKER20_precipType', 'MARKER20_pressure', 'MARKER20_temperatureMax', 'MARKER20_temperatureMin', 'MARKER20_visibility',
                       'MARKER20_windBearing', 'MARKER20_windGust', 'MARKER20_windSpeed', 'OUTAGE_DURATION', 'OUTAGED_RECOVERY',
                       'CUST_QTY', 'NO_OF_OUTAGES']]

In [21]:
final_ads.isnull().sum()

MAJ_OTG_ID                          0
MARKER1_apparentTemperatureMax      0
MARKER1_apparentTemperatureMin      0
MARKER1_cloudCover                  0
MARKER1_dewPoint                    0
MARKER1_humidity                    0
MARKER1_icon                        0
MARKER1_precipIntensity             0
MARKER1_precipIntensityMax          0
MARKER1_precipProbability           0
MARKER1_precipType                  5
MARKER1_pressure                    0
MARKER1_temperatureMax              0
MARKER1_temperatureMin              0
MARKER1_visibility                  0
MARKER1_windBearing                 0
MARKER1_windGust                    0
MARKER1_windSpeed                   0
MARKER2_apparentTemperatureMax      0
MARKER2_apparentTemperatureMin      0
MARKER2_cloudCover                  0
MARKER2_dewPoint                    0
MARKER2_humidity                    0
MARKER2_icon                        0
MARKER2_precipIntensity             0
MARKER2_precipIntensityMax          0
MARKER2_prec

## **WRITE TO CSV**

In [22]:
final_ads.to_csv("gs://aes-datahub-0001-curated/Outage_Restoration/Historical_Data/Storm_Dataset/OMS_IPL_STORM_DATASET_V1.0.csv")